In [ ]:
from thunderpack import ThunderReader

reader = ThunderReader('/storage/vbutoi/datasets/MSD/old_thunder_files/HepaticVessel/CT/0')

In [ ]:
type(reader)

In [ ]:
# list(reader.keys())

In [ ]:
list(reader['_splits'].keys())

In [ ]:
from typing import List, Tuple
from sklearn.model_selection import train_test_split

def data_splits(
    values: List[str], 
    splits: Tuple[float, float, float, float], 
    seed: int
) -> Tuple[List[str], List[str], List[str], List[str]]:

    if len(set(values)) != len(values):
        raise ValueError(f"Duplicate entries found in values")

    # Super weird bug, removing for now, add up to 1!
    # if (s := sum(splits)) != 1.0:
    #     raise ValueError(f"Splits must add up to 1.0, got {splits}->{s}")

    train_size, cal_size, val_size, test_size = splits
    values = sorted(values)
    # First get the size of the test splut
    traincalval, test = train_test_split(values, test_size=test_size, random_state=seed)
    # Next size of the val split
    val_ratio = val_size / (train_size + cal_size + val_size)
    traincal, val = train_test_split(traincalval, test_size=val_ratio, random_state=seed)
    # Next size of the cal split
    cal_ratio = cal_size / (train_size + cal_size)
    train, cal = train_test_split(traincal, test_size=cal_ratio, random_state=seed)

    assert sorted(train + cal + val + test) == values, "Missing Values"

    return (train, cal, val, test)

In [ ]:
import os

def find_folders_with_mdb_files(root_dir):
    matching_folders = []

    for dirpath, dirnames, filenames in os.walk(root_dir):
        if 'data.mdb' in filenames and 'lock.mdb' in filenames:
            matching_folders.append(dirpath)

    return matching_folders

# # Replace 'old_file_root' with the path to your root directory
# old_file_root = '/storage/vbutoi/datasets/MSD/old_thunder_files'
# thunder_folders = find_folders_with_mdb_files(old_file_root)

# # Print the paths of matching folders
# for folder in thunder_folders:
#     print(folder)

In [ ]:
from thunderpack import ThunderDB

# Replace 'old_file_root' with the path to your root directory
old_file_root = '/storage/vbutoi/datasets/MSD/old_thunder_files'
thunder_folders = find_folders_with_mdb_files(old_file_root)

old_prefix = "/storage/vbutoi/datasets/MSD/old_thunder_files"
new_folder_prefix = "/storage/vbutoi/datasets/MSD/thunder_MSD/v4.2/MSD"
splits_seed = 42
splits_ratio = (0.6, 0.2, 0.1, 0.1)

for old_thunder_path in thunder_folders:
    new_path = old_thunder_path.replace(old_prefix, new_folder_prefix)
    old_db_obj = ThunderReader(old_thunder_path)
    print("Task:", old_thunder_path, "Num Examples:", len(old_db_obj['_subjects']))
    # If the new path folder doesn't exist, create it
    # if not os.path.exists(new_path):
    #     os.makedirs(new_path)
    # print("Moved:", old_thunder_path, "->", new_path)
    # ### Make a new ThunderDB object that is identical to the old one, but with new splits.
    # with ThunderDB.open(str(new_path), "c") as new_db:
    #     for key in old_db_obj.keys():
    #         if key != "_splits":
    #             new_db[key] = old_db_obj[key]
    #     # Redo the splits! 
    #     subjects = sorted(new_db['_subjects'])
    #     splits = data_splits(subjects, splits_ratio, splits_seed)
    #     # Finally, put the new splits in
    #     new_db['_splits'] = dict(zip(("train", "cal", "val", "test"), splits))